In [1]:
%pip install transformers
%pip install tf-keras
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


### Load libraries

In [2]:
from google.cloud import bigquery
from multiprocessing import Pool
import torch

### Read Original Data

In [3]:
def read_bq():
    client = bigquery.Client()
    
    query = f"""
        SELECT *
        FROM `intern-project-415606.Criminal_Dataset.criminal_data`
    """
    
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        return df
    except Exception as e:
        print(f"Error: {e}")

In [4]:
df = read_bq()

In [5]:
df

,JLR_LINK,TRANS_TYPE_OF_CASE,TRANS_LEGAL_RELATIONSHIP,PDF_TEXT,EXTRACT,ID,NAME,Year,Month,Day,GENDER,BIRTH
0,https://congbobanan.toaan.gov.vn/2ta827827t1cv...,採取行政處理措施的決定,放入強制性戒毒機構,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,Họ và tên: Nguyễn Văn C – Giới tính: Nam Sinh...,No_Id,Nguyễn Văn C,1994,1,25,Male,1994-01-25 00:00:00
1,https://congbobanan.toaan.gov.vn/2ta889668t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Họ và tên: Trần Ngọc T, sinh ngày 08/12/1993....",No_Id,Trần Ngọc T,1993,12,8,Male,1993-12-08 00:00:00
2,https://congbobanan.toaan.gov.vn/2ta153804t1cv...,刑事,盜竊財產罪,<Page:1>TOÀ ÁN NHÂN DÂN CỘNG HOÀ XÃ HỘI CHỦ NG...,"Bùi Văn T, sinh ngày 09/12/1989 Tại: Xóm T – ...",No_Id,Bùi Văn T,1989,12,9,Male,1989-12-09 00:00:00
3,https://congbobanan.toaan.gov.vn/2ta1126341t1c...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Nguyễn Văn Đ, sinh ngày 04/8/1978 tại huyện H...",No_Id,Nguyễn Văn Đ,1978,8,4,Male,1978-08-04 00:00:00
4,https://congbobanan.toaan.gov.vn/2ta1066372t1c...,刑事,欺詐佔有財產罪,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"Cao Thị B, sinh ngày 01/01/1975, tại huyện C,...",No_Id,Cao Thị B,1975,1,1,Female,1975-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
36545,https://congbobanan.toaan.gov.vn/2ta465562t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,Họ và tên: Trần Thành M; Tên gọi khác: Kh«ng;...,No_Id,Trần Thành M,1977,4,16,Male,1977-04-16 00:00:00
36546,https://congbobanan.toaan.gov.vn/2ta938857t1cv...,刑事,犯罪非法持有毒品,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NG...,"1. Nguyễn Đình T, sinh năm 1997 tại xã TK, hu...",No_Id,Nguyễn Đình Q,2005,9,14,Male,2005-09-14 00:00:00
36547,https://congbobanan.toaan.gov.vn/2ta107179t1cv...,刑事,故意對他人造成傷害或損害健康的罪行,<Page:1>TÒA ÁN NHÂN DÂN CỘNG HOÀ XÃ HỘI CHỦ NG...,"1- Huỳnh Duy Kh ( tức Kh), sinh ngày 15/7/199...",No_Id,Huỳnh Duy Kh,1997,7,15,Male,1997-07-15 00:00:00
36548,https://congbobanan.toaan.gov.vn/2ta1064276t1c...,刑事,盜竊財產罪,<Page:1>TÒA ÁN QUÂN SỰ CỘNG HÒA XÃ HỘI CHỦ NGH...,"1. Hồ Văn C, sinh ngày 30/7/2003 tại tỉnh Quả...",No_Id,Hồ Văn X,2003,3,6,Male,2003-03-06 00:00:00


### Use transformer (electra-based-ner-vn)

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="NlpHUST/ner-vietnamese-electra-base")

2024-05-15 06:05:46.220804: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 06:05:46.225404: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 06:05:46.290751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# test out
# the transformer only accept 512 tokens, need to separate them into chunks
def ner(text):
    return pipe(text)

def process_text_chunks_by_index(text, chunk_size=512):
    # Initialize an empty list to store the results
    results = []
    
    # Calculate the total number of chunks
    num_chunks = (len(text) + chunk_size - 1) // chunk_size
    
    # Process each chunk of text
    for i in range(num_chunks):
        # Calculate the start and end indices for the current chunk
        start_index = i * chunk_size
        end_index = min((i + 1) * chunk_size, len(text))
        
        # Extract the current chunk of text
        chunk_text = text[start_index:end_index]
        
        chunk_result = ner(chunk_text)
        
        # Append the chunk to the results
        results.append(chunk_result)
    # Return the list of results
    return results

# Example usage:
# Assuming df['PDF_TEXT'][0] contains your text data
text = df['PDF_TEXT'][0]

# Process text chunks
results = process_text_chunks_by_index(text, 512)

# Now, 'results' will contain a list of results for each chunk of text
for result in results[0]:
    if 'PERSON' in result['entity']:
        print(result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entity': 'I-PERSON', 'score': 0.5714736, 'index': 40, 'word': 'Sơn', 'start': 133, 'end': 136}
{'entity': 'B-PERSON', 'score': 0.6212477, 'index': 74, 'word': 'TỈNH', 'start': 273, 'end': 277}
{'entity': 'I-PERSON', 'score': 0.4203355, 'index': 76, 'word': 'AN', 'start': 283, 'end': 285}
{'entity': 'B-PERSON', 'score': 0.76542866, 'index': 92, 'word': 'Ông', 'start': 343, 'end': 346}
{'entity': 'B-PERSON', 'score': 0.99588686, 'index': 93, 'word': 'Phan', 'start': 347, 'end': 351}
{'entity': 'I-PERSON', 'score': 0.99911934, 'index': 94, 'word': 'Văn', 'start': 352, 'end': 355}
{'entity': 'I-PERSON', 'score': 0.99886405, 'index': 95, 'word': 'Phú', 'start': 356, 'end': 359}
{'entity': 'B-PERSON', 'score': 0.88995075, 'index': 103, 'word': 'Ông', 'start': 381, 'end': 384}
{'entity': 'B-PERSON', 'score': 0.99539137, 'index': 104, 'word': 'Phạm', 'start': 385, 'end': 389}
{'entity': 'I-PERSON', 'score': 0.9992173, 'index': 105, 'word': 'Mạnh', 'start': 390, 'end': 394}
{'entity': 'I-PERS

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base", output_hidden_states=True)

# Encode some text
input_text = df['PDF_TEXT'][0][:512]
inputs = tokenizer(input_text, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states[-1]

# `embeddings` now contains the embeddings of the input text
# Find the token index for 'nam'
word = "Phan"
word_tokens = tokenizer.tokenize(word)
word_ids = tokenizer.convert_tokens_to_ids(word_tokens)

# Find the position of 'nam' in the input text
input_ids = inputs["input_ids"][0].tolist()
word_index = input_ids.index(word_ids[0])

# Extract the vector for 'nam'
word_vector = hidden_states[0, word_index]

print(len(word_vector))
print(word_vector)

768
tensor([ 5.1100e-01, -7.9762e-01, -1.0493e-01, -4.8929e-01,  6.3032e-02,
         9.6179e-02, -2.6717e-01,  3.0017e-01,  2.7119e-01, -5.2590e-02,
        -3.5478e-01,  1.0288e+00, -3.2004e-02, -5.0650e-01,  9.1852e-01,
        -6.2743e-01,  2.8712e-01, -3.5392e-01,  1.1302e-01, -2.3140e-01,
        -5.6843e-01,  5.9286e-01,  1.6639e-01, -1.8746e-01,  1.0147e+00,
         1.8835e-01,  5.0857e-01, -9.0540e-01,  1.0142e+00,  4.8313e-01,
         5.9206e-02,  8.2913e-01, -3.4408e-01, -3.4592e-01, -1.6578e-02,
        -5.0661e-01,  5.2788e-01,  6.0776e-01,  9.0973e-01, -8.4370e-02,
        -6.2213e-01,  7.0673e-02, -2.7563e-01, -2.7551e-01,  5.5595e-01,
        -2.1407e-01,  6.6735e-01,  7.1493e-02,  1.2762e-01,  4.4231e-01,
        -8.8343e-01,  5.3612e-01,  3.2673e-01,  4.4131e-01, -4.3692e-01,
         6.3926e-02, -2.8878e-01,  3.8481e-01, -4.9923e-01,  3.7791e-01,
         5.6250e-01,  8.8277e-01,  3.8190e-01,  5.3631e-01, -7.2991e-01,
        -3.8005e-01, -2.1037e-01,  2.5419e-01, 

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

# Example text
aa = df['PDF_TEXT'][0]
aa = aa[:512]
# Tokenize text
encoded_input = tokenizer.encode(aa, add_special_tokens=True)

# Tokens to words mapping
tk = tokenizer.convert_ids_to_tokens(encoded_input)

print(len(tk))
print(aa)
print(tk)

138
<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
HUYỆN ANH SƠN Độc lập – Tự do – Hạnh phúc
TỈNH NGHỆ AN
Số: 17/QĐ – TA Anh Sơn, ngày 18 tháng 5 năm 2021
QUYẾT ĐỊNH
Áp dụng biện pháp xử lý hành chính đưa và cơ sở cai nghiện bắt buộc
TÒA ÁN NHÂN DÂN HUYỆN ANH SƠN, TỈNH NGHỆ AN
Với thành phần tiến hành phiên họp gồm có:
1. Thẩm phán: Ông Phan Văn Phú
2. Thư ký phiên họp: Ông Phạm Mạnh Cường
Ngày 18 tháng 5 năm 2021, tại trụ sở Tòa án nhân dân huyện A, tỉnh N tiến
hành phiên họp xem xét, quyết định áp
['[CLS]', '[UNK]', 'Page', ':', '1', '[UNK]', '[UNK]', 'ÁN', 'NHÂN', 'DÂN', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'NAM', '[UNK]', 'ANH', '[UNK]', 'Độc', 'lập', '–', 'Tự', 'do', '–', 'Hạnh', 'phúc', '[UNK]', '[UNK]', 'AN', 'Số', ':', '17', '/', 'QĐ', '–', 'TA', 'Anh', 'Sơn', ',', 'ngày', '18', 'tháng', '5', 'năm', '2021', '[UNK]', '[UNK]', 'Áp', 'dụng', 'biện', 'pháp', 'xử', 'lý', 'hành', 'chính', 'đưa', 'và', 'cơ', 'sở', 'cai', 'nghiện', 'bắt', 'buộc', '

In [19]:
import re
# Split the text using regular expression to handle both space and newline characters
tokens = re.split(r'\s+', df['PDF_TEXT'][0])

# Filter out empty tokens
tokens = [token.strip() for token in tokens if token.strip()]

#check chunk size (256 per chunk)
chunk_size = (len(tokens) // 256) + 1
chunk_result = []
char_index = 0
for i in range(chunk_size):
    chunk_text = " ".join(tokens[i*256:min((i+1)*256, len(text))])
    chunk_ner = pipe(chunk_text)
    for ner in chunk_ner:
        ner['start'] += char_index 
    chunk_result.extend(chunk_ner)
    char_index += len(chunk_text)
# for cr in chunk_result:
#     print(cr)

chunk_result_dict = {item['start']: item['entity'] for item in chunk_result}
chunk_result[0]

{'entity': 'I-ORGANIZATION',
 'score': 0.7754121,
 'index': 8,
 'word': 'NHÂN',
 'start': 15,
 'end': 19}

In [36]:
sequence = 1
rows = []
char_index = 0
for t in range(len(tokens)):

    if char_index not in chunk_result_dict.keys():
        row = {
            "extract_id": i,  # You can use the index 'i' as the ID for each sentence
            "text": tokens[t],
            "ner_tag": 'other',
            "sequence": sequence
        }
                              
    else:
        row = {
            "extract_id": i,  # You can use the index 'i' as the ID for each sentence
            "text": tokens[t],
            "ner_tag": chunk_result_dict[char_index],
            "sequence": sequence
        }
    sequence += 1
    rows.append(row)
    char_index += len(tokens[t]) + 1

<Page:1>TÒA ÁN NHÂN DÂN CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
HUYỆN ANH SƠN Độc lập – Tự do – Hạnh phúc
TỈNH NGHỆ AN
Số: 17/QĐ – TA Anh Sơn, ngày 18 tháng 5 năm 2021
QUYẾT ĐỊNH
Áp dụng biện pháp xử lý hành chính đưa và cơ sở cai nghiện bắt buộc
TÒA ÁN NHÂN DÂN HUYỆN ANH SƠN, TỈNH NGHỆ AN
Với thành phần tiến hành phiên họp gồm có:
1. Thẩm phán: Ông Phan Văn Phú
2. Thư ký phiên họp: Ông Phạm Mạnh Cường
Ngày 18 tháng 5 năm 2021, tại trụ sở Tòa án nhân dân huyện A, tỉnh N tiến
hành phiên họp xem xét, quyết định áp dụng biện pháp xử lý hành chính đưa vào cơ
sở cai nghiện bắt buộc theo Quyết định mở phiên họp số 17/QĐ – TA ngày 12 tháng
5 năm 2021 đối với:
Họ và tên: Nguyễn Văn C – Giới tính: Nam
Sinh ngày: 25/01/1994
Dân tộc: Kinh; Tôn giáo: Thiên chúa giáo; Trình độ văn hóa: 9/12
Nơi cư trú: thôn 12, xã T, huyện A, tỉnh N
Nghề nghiệp: Lao động tự do
Tiền án, tiền sự: Không
Con ông: Nguyễn Văn Trung; con bà: Nguyễn Thị Hà.
Có mặt tại phiên họp
Có sự tham gia của:
1. Đại diện cơ quan đề nghị: Ô